In [1]:
%matplotlib inline
import importlib
from pytorch_utils_oh_1 import *

Pytorch utils oh: pytorch_utils_oh_1.py
Pytorch: 0.2.0_4


In [2]:
MODEL_SAVE_PATH = 'letters_1'

import pytorch_utils_oh_1; importlib.reload(pytorch_utils_oh_1); from pytorch_utils_oh_1 import *;

# Data loading

In [145]:
all_data = pickle.load(open("data/en_train_not_changed_verb_fix_2.pkl", "rb" ))
all_data_sentence_index = all_data.set_index('sentence_id')

In [146]:
letters_data = all_data[all_data['class']=='LETTERS']

In [147]:
letters_data.sample(10)

,sentence_id,token_id,class,before,after
6909787,524750,9,LETTERS,D.C.'s,d c's
3847166,295812,13,LETTERS,PDF,p d f
2422233,188158,3,LETTERS,cnn,c n n
8657415,654804,6,LETTERS,Ph,p h
6896852,523780,12,LETTERS,U.S.,u s
1242585,97936,7,LETTERS,US,u s
1934541,150902,7,LETTERS,FM,f m
8449318,639418,15,LETTERS,XVIIe,x v i i e
2495561,193732,13,LETTERS,U.K.,u k
6745899,512597,18,LETTERS,P.,p


In [148]:
letters_match_re = re.compile(r"[a-zA-Z]") 
def letters_transform_baseline(string):
    global letters_before_after_last_string
    letters_before_after_last_string = string
    arr = []
    if string == "'s":
        return string
    for i, c in enumerate(string):
        if letters_match_re.match(c):
            arr.append(c)
        elif c == '&':
            arr.append('and')
        elif len(string) > 1 and i==len(string)-2 and string[-1] == 's':
            arr[-1] = arr[-1] + "'s"
            break
        elif len(string) > 2 and i==len(string)-3 and string[-2:] == 's':
            arr[-1] = arr[-1] + "'s"
        elif c == 'é':
            arr.append('e')
            arr.append('acute')
            
    return ' '.join(arr).lower()

In [149]:
letters_transform_baseline("AE's")

"a e's"

In [150]:
def eval_baseline(n_sample=20, print_wrongs=True):
    n_correct = 0 
    for _ in range(n_sample):
        row = letters_data.iloc[random.randint(1, len(letters_data)-1)]
        tmp=row
        s_bef = row['before']
        s_aft = row['after']
        result = letters_transform_baseline(s_bef)
        if result == s_aft:
            n_correct += 1
            # print("YAY: {} => {}".format(s_bef, s_aft))
        elif print_wrongs:
            print("NOO: {} => {} ({})".format(s_bef, result, s_aft))
    print("Accuracy: {:>4.2%} ({:>8d}/{:>8d})".format(n_correct/n_sample, n_correct, n_sample))
        

In [119]:
eval_baseline(1000)

NOO: CDs => c d s (c d's)
NOO: GCSEs => g c s e s (g c s e's)
NOO: Ulus => u l u s (u l u's)
NOO: Thos => t h o s (t h o's)
NOO: NOCs => n o c s (n o c's)
NOO: ATVs => a t v s (a t v's)
NOO: Québécois => q u e acute b e acute c o i s (q u e acute b e acute c o i's)
NOO: Tas => t a s (t a's)
NOO: Ves => v e s (v e's)
NOO: SDKs => s d k s (s d k's)
NOO: OKs => o k s (o k's)
NOO: Mes => m e s (m e's)
Accuracy: 98.80% (     988/    1000)


In [114]:
eval_baseline(100000, print_wrongs=False)

Accuracy: 98.59% (   98589/  100000)


In [4]:
all_data.sample(5)

,sentence_id,token_id,class,before,after
7459959,565730,6,NOT_CHANGED,",",","
4751479,363745,3,NOT_CHANGED,Hispanic,Hispanic
2165066,168526,16,NOT_CHANGED,.,.
6848805,520203,1,NOT_CHANGED,applications,applications
5290088,404069,4,NOT_CHANGED,Cup,Cup


In [5]:
categories_all = all_data["class"].unique()
print(categories_all)
print(len(categories_all))
categories_index = dict((c, i) for i, c in enumerate(categories_all))

['NOT_CHANGED' 'NUMBERS' 'LETTERS' 'PLAIN' 'VERBATIM' 'ELECTRONIC']
6


### Letters all

In [9]:
tmp = sorted(list(set(''.join(all_data['before']))))
characters_all = ['<EOS>', '<SOS>'] + sorted(list(set(tmp)))
characters_all_index = dict((c, i) for i, c in enumerate(characters_all))
print(''.join(characters_all))
print(len(characters_all))

<EOS><SOS> !"#$%&'()+,-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWXYZ[]_abcdefghijklmnopqrstuvwxyz|~¡£¥ª«²³µº»¼½¾¿éɒʻˈΩμ—€⅓⅔⅛⅝⅞☒
115


### Common words

In [10]:
common_words = pickle.load(open("data/en_train_words_before_over_100.pk", "rb" ))
common_words = [EOS_TOKEN, SOS_TOKEN, UNKNOWN_WORD_TOKEN, NUMBER_WORD_TOKEN] + common_words
common_words_index = dict((c, i) for i, c in enumerate(common_words))
len(common_words)

7380

### More balanced sample

In [6]:
all_data.groupby("class")["class"].count()

class
ELECTRONIC        4964
LETTERS         144364
NOT_CHANGED    9218584
NUMBERS         448151
PLAIN            36472
VERBATIM         65855
Name: class, dtype: int64

In [254]:
balanced_data_classes_select = list(all_data.groupby('class'))
balanced_data_accessed_counter = 0 
balanced_data_length = 0
def balanced_data_randomize(max_len=20000):
    global balanced_data, balanced_data_length, balanced_data_accessed_counter
    balanced_data = pd.concat([v.sample(min(max_len, len(v))) for k, v in balanced_data_classes_select])
    balanced_data_length = len(balanced_data)
    balanced_data_accessed_counter = 0

def balanced_data_sample_row():
    global balanced_data_accessed_counter
    balanced_data_accessed_counter += 1
    if balanced_data_accessed_counter/balanced_data_length > 0.2:
        balanced_data_randomize()
    return balanced_data.iloc[random.randint(1, balanced_data_length-1)]
    
balanced_data_randomize()

In [224]:
balanced_data.groupby("class")["class"].count()

class
ELECTRONIC      4964
LETTERS        20000
NOT_CHANGED    20000
NUMBERS        20000
PLAIN          20000
VERBATIM       20000
Name: class, dtype: int64

### Samples

In [226]:
def get_random_sample():
    sample_row = balanced_data_sample_row()
    sentence_id = sample_row['class']

    rows = all_data_sentence_index.loc[sample_row['sentence_id']]
    befores = list(rows.before)
        
    token_id_idx = list(rows['token_id']).index(sample_row['token_id'])
    befores[token_id_idx] = SAMPLE_WORD_TOKEN
    
    return sample_row['before'], sample_row['after'], sample_row['class'], befores
            
def tmp():
    s_bef, s_aft, s_class, s_sentence = get_random_sample()
    print(s_class, ':', s_bef, '->', s_aft)
    print(' '.join(s_sentence))
    print(s_sentence)
    print(words_to_tensor(list(s_sentence), common_words_index).shape)
    print(string_to_tensor(s_bef, characters_all_index).shape)
tmp()

ELECTRONIC : Catholic-Hierarchy.org -> c a t h o l i c d a s h h i e r a r c h y dot o r g
" Bishop Giuseppe Pamphilj ( Panfili ) , O.S.A. " <SAMPLE> .
['"', 'Bishop', 'Giuseppe', 'Pamphilj', '(', 'Panfili', ')', ',', 'O.S.A.', '"', '<SAMPLE>', '.']
torch.Size([1, 13, 7380])
torch.Size([1, 23, 115])
